In [1]:
import pandas as pd
import sqlite3

In [2]:
df = pd.read_csv('cleaned_students_data.csv')

In [3]:
df.head()

,student_id,name,subject,score,date,attendance,performance_category
0,109,Janice Ortiz,Computer,72.0,2024-10-25,35,Medium
1,156,Curtis Rivera,English,94.0,2025-06-30,40,High
2,177,Chad Diaz,History,100.0,2025-09-19,21,High
3,132,Aaron Wilson,History,53.0,2024-11-26,33,Low
4,160,Danielle Barron,Computer,68.0,2025-04-18,24,Low


In [4]:
conn = sqlite3.connect('students_performances.db')

In [5]:
cur = conn.cursor()

In [6]:
cur.executescript("""

    DROP TABLE IF EXISTS StagingStudentPerformance;
    
    CREATE TABLE StagingStudentPerformance (
    student_id            INTEGER,
    name                  TEXT,
    subject               TEXT,
    score                 REAL,
    date                  TEXT,
    attendance            INTEGER,
    performance_category  TEXT
    );

    """) 

In [7]:
cur.executescript("""
    CREATE TABLE IF NOT EXISTS dim_students (
        student_sk INTEGER PRIMARY KEY AUTOINCREMENT,
        student_id INTEGER,
        student_name TEXT NOT NULL
    );

    CREATE TABLE IF NOT EXISTS dim_subject (
        subject_sk INTEGER PRIMARY KEY AUTOINCREMENT,
        subject_name TEXT
    );

    CREATE TABLE IF NOT EXISTS dim_performance_category (
        performance_sk   INTEGER PRIMARY KEY AUTOINCREMENT,
        performance_category_name TEXT NOT NULL
    );

    CREATE TABLE IF NOT EXISTS dim_date (
        date_sk INTEGER PRIMARY KEY AUTOINCREMENT,
        date TEXT,
        year INTEGER,
        month INTEGER,
        day INTEGER,
        month_name TEXT,
        quarter INTEGER
    );

    CREATE TABLE IF NOT EXISTS fact_student_performance (
        fact_id_sk INTEGER PRIMARY KEY AUTOINCREMENT,
        score INTEGER,
        attendance INTEGER,
        student_fk INTEGER,
        subject_fk INTEGER,
        date_fk INTEGER,
        performance_fk INTEGER,
        FOREIGN KEY(student_fk) REFERENCES dim_students(student_sk),
        FOREIGN KEY(subject_fk) REFERENCES dim_subject(subject_sk),
        FOREIGN KEY(date_fk) REFERENCES dim_date(date_sk),
        FOREIGN KEY(performance_fk) REFERENCES dim_performance_category(performance_sk)
    );
    """)

conn.commit()

In [8]:
cur = conn.cursor()

df["student_id"] = df["student_id"].astype(str).str.strip()
df["student_name"] = df["name"].astype(str).str.strip()
df["subject"] = df["subject"].astype(str).str.strip()
df["performance_category"] = df["performance_category"].astype(str).str.strip()
df["date"] = pd.to_datetime(df["date"]).dt.strftime("%Y-%m-%d")
df["score"] = df["score"].astype(int)
df["attendance"] = df["attendance"].astype(int)
conn.commit()

In [9]:
df.to_sql("staging_student_performance", conn, if_exists="replace", index=False)
conn.commit()

In [10]:
cur.execute("""
INSERT INTO dim_students (student_id, student_name)
SELECT DISTINCT student_id, student_name
FROM staging_student_performance;
""")
conn.commit()

In [11]:
cur.execute("""
INSERT INTO dim_subject (subject_name)
SELECT DISTINCT subject
FROM staging_student_performance;
""")
conn.commit()

In [12]:
cur.execute("""
INSERT INTO dim_performance_category (performance_category_name)
SELECT DISTINCT performance_category
FROM staging_student_performance;
""")
conn.commit()

In [13]:
cur.execute("""
INSERT INTO dim_date (date, year, month, day, month_name, quarter)
SELECT DISTINCT 
    date,
    CAST(strftime('%Y', date) AS INTEGER),
    CAST(strftime('%m', date) AS INTEGER),
    CAST(strftime('%d', date) AS INTEGER),
    strftime('%m', date), 
    CAST((strftime('%m', date) - 1) / 3 + 1 AS INTEGER)
FROM staging_student_performance;
""")
conn.commit()

In [14]:
cur.execute("""
INSERT INTO fact_student_performance (
    score, attendance, student_fk, subject_fk, date_fk, performance_fk
)
SELECT 
    stg.score,
    stg.attendance,
    s.student_sk,
    sub.subject_sk,
    d.date_sk,
    p.performance_sk
FROM staging_student_performance stg
JOIN dim_students s ON s.student_id = stg.student_id
JOIN dim_subject sub ON sub.subject_name = stg.subject
JOIN dim_date d ON d.date = stg.date
JOIN dim_performance_category p ON p.performance_category_name = stg.performance_category;

""")
conn.commit()

In [15]:
statement ="SELECT * FROM dim_students"
cur.execute(statement)
  
output1 = cur.fetchmany(2)
for row in output1:
  print(row)

(1, 109, 'Janice Ortiz')
(2, 156, 'Curtis Rivera')


In [16]:
cur.execute("SELECT COUNT(*) FROM fact_student_performance;")
print(cur.fetchone())


(7232,)


In [19]:
top_students ="""SELECT s.student_name,
                 sub.subject_name,
                 f.score
                 FROM fact_student_performance f
                 JOIN dim_students s ON f.student_fk = s.student_sk
                 JOIN dim_subject sub ON f.subject_fk = sub.subject_sk
                 WHERE f.score = (
                     SELECT MAX(f2.score)
                     FROM fact_student_performance f2
                     WHERE f2.subject_fk = f.subject_fk
                 );
"""
cur.execute(top_students)
  
output1 = cur.fetchmany(10)
for row in output1:
  print(row)

('Chad Diaz', 'History', 100)
('Charles Brown', 'History', 100)
('Janet Bryant', 'History', 100)
('Cindy Santana', 'History', 100)
('Michelle Norman', 'History', 100)
('Todd George', 'History', 100)
('Chad Diaz', 'History', 100)
('Charles Brown', 'History', 100)
('Janet Bryant', 'History', 100)
('Cindy Santana', 'History', 100)


In [29]:
attendance = """
                SELECT 
                    strftime('%Y-%m', d.date) AS month, 
                    AVG(f.attendance) AS avg_attendance
                FROM fact_student_performance f
                JOIN dim_date d ON f.date_fk = d.date_sk
                GROUP BY month
                ORDER BY month;
"""
cur.execute(attendance)
  
output2 = cur.fetchmany(10)
for row in output2:
  print(row)

('2024-10', 27.936026936026938)
('2024-11', 31.647959183673468)
('2024-12', 31.16591928251121)
('2025-01', 28.91810344827586)
('2025-02', 29.726315789473684)
('2025-03', 29.08076923076923)
('2025-04', 29.0)
('2025-05', 29.89119170984456)
('2025-06', 30.562130177514792)
('2025-07', 29.20138888888889)


In [30]:
average_score_per_subject="""
SELECT sub.subject_name,
       ROUND(AVG(f.score), 2) AS avg_score
FROM fact_student_performance f
JOIN dim_subject sub ON f.subject_fk = sub.subject_sk
GROUP BY sub.subject_name
ORDER BY avg_score DESC;
"""
cur.execute(average_score_per_subject)
output1 = cur.fetchall()
for row in output1:
  print(row)

('English', 73.57)
('Science', 73.47)
('Geography', 71.24)
('History', 70.92)
('Computer', 70.5)
('Physics', 70.04)
('Chemistry', 69.96)
('Math', 69.8)
('Biology', 67.89)


In [33]:
Attendance_Score="""
SELECT sub.subject_name,
       ROUND(AVG(f.attendance), 2) AS avg_attendance,
       ROUND(AVG(f.score), 2) AS avg_score
FROM fact_student_performance f
JOIN dim_subject sub ON f.subject_fk = sub.subject_sk
GROUP BY sub.subject_name;

"""
cur.execute(Attendance_Score)
output1 = cur.fetchall()
for row in output1:
  print(row)

('Biology', 30.17, 67.89)
('Chemistry', 30.52, 69.96)
('Computer', 30.9, 70.5)
('English', 29.67, 73.57)
('Geography', 30.18, 71.24)
('History', 30.15, 70.92)
('Math', 29.72, 69.8)
('Physics', 30.85, 70.04)
('Science', 28.38, 73.47)
